<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-dependencies" data-toc-modified-id="Load-dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load dependencies</a></span></li><li><span><a href="#Read-csv-files-to-DFs" data-toc-modified-id="Read-csv-files-to-DFs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read csv files to DFs</a></span></li><li><span><a href="#Fill-missing-values" data-toc-modified-id="Fill-missing-values-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fill missing values</a></span></li><li><span><a href="#Add-date-part" data-toc-modified-id="Add-date-part-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Add date part</a></span></li><li><span><a href="#Add-extra-columns-(purchased-on-weekend,-monthend,-month_diff-etc." data-toc-modified-id="Add-extra-columns-(purchased-on-weekend,-monthend,-month_diff-etc.-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Add extra columns (purchased on weekend, monthend, month_diff etc.</a></span></li><li><span><a href="#Mark-as-categorical-variables" data-toc-modified-id="Mark-as-categorical-variables-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Mark as categorical variables</a></span></li><li><span><a href="#One-hot-encoding-categories-(not-working)" data-toc-modified-id="One-hot-encoding-categories-(not-working)-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>One hot encoding categories (not working)</a></span></li><li><span><a href="#Aggregate-by-card_id" data-toc-modified-id="Aggregate-by-card_id-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Aggregate by card_id</a></span></li><li><span><a href="#Add-exta-interpreted-columns-on-aggregates" data-toc-modified-id="Add-exta-interpreted-columns-on-aggregates-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Add exta interpreted columns on aggregates</a></span></li><li><span><a href="#Load-test-&amp;-train-DFs" data-toc-modified-id="Load-test-&amp;-train-DFs-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Load test &amp; train DFs</a></span></li><li><span><a href="#Add-date-part-to-test-&amp;-train-dfs" data-toc-modified-id="Add-date-part-to-test-&amp;-train-dfs-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Add date part to test &amp; train dfs</a></span></li><li><span><a href="#Merge-train-&amp;-test-with-new-&amp;-old-transactions-history" data-toc-modified-id="Merge-train-&amp;-test-with-new-&amp;-old-transactions-history-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Merge train &amp; test with new &amp; old transactions history</a></span></li><li><span><a href="#Add-extra-columns-like-age,-total-transactions,-installments,-purchase-amount,-first-buy-etc" data-toc-modified-id="Add-extra-columns-like-age,-total-transactions,-installments,-purchase-amount,-first-buy-etc-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Add extra columns like age, total transactions, installments, purchase amount, first buy etc</a></span></li><li><span><a href="#Mark-the-outliers" data-toc-modified-id="Mark-the-outliers-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Mark the outliers</a></span></li><li><span><a href="#Target-encode-the-outliers" data-toc-modified-id="Target-encode-the-outliers-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Target encode the outliers</a></span></li></ul></div>

1. Load dependencies
2. Read csv files to dataframes
3. Fill missing values
4. Add date part 
5. Add extra columns (purchased on weekend, monthend, month_diff etc.
6. Aggregate by card_id
7. Aggregate by categories
8. Mark categorical columns
9. Add exta interpreted columns on aggregates
10. Load test & train csvs to dfs
11. Add date part to test & train dfs
12. Merge train & test with new & old transactions history
13. Add extra columns like age, total transactions, installments, purchase amount, first buy etc
14. Mark the outliers
15. Target encode the outliers 
16. Save to feather

### Load dependencies

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

Autosaving every 120 seconds


In [2]:
from fastai.io import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
import feather

### Read csv files to DFs

In [3]:
PATH = '../data/elo/'

In [4]:
files = ['historical_transactions', 'new_merchant_transactions']

In [5]:
hist_trans, new_hist_trans = [pd.read_csv(f'{PATH}{c}.csv') for c in files]

### Fill missing values

In [6]:
def fill_nas_for_transactions_df(df):
    # Fill nas for category_3 with mode
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    return df

In [7]:
dfs = [hist_trans, new_hist_trans]

In [8]:
hist_trans, new_hist_trans = [fill_nas_for_transactions_df(df) for df in dfs]

### Add date part

In [9]:
add_datepart(hist_trans, 'purchase_date', drop=False)
add_datepart(new_hist_trans, 'purchase_date', drop=False)

### Add extra columns (purchased on weekend, monthend, month_diff etc.

In [10]:
def add_extra_cols(df):
    df['purchased_on_weekend'] = (df.purchase_Dayofweek >=5).astype(int)
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    return df

In [11]:
hist_trans, new_hist_trans = [add_extra_cols(df) for df in dfs]

In [12]:
hist_trans.shape, new_hist_trans.shape

((29112361, 29), (1963031, 29))

### Mark as categorical variables

In [13]:
def mark_categorical_cols_for_transactions(df):
    categorical_cols = ['authorized_flag', 'card_id', 'city_id',
       'merchant_category_id', 'merchant_id',
       'state_id', 'subsector_id', 'purchase_Year', 'purchase_Month']
    for c in categorical_cols:
        df[c] = df[c].astype('category').cat.as_ordered()
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

In [14]:
hist_trans_c, new_hist_trans_c = [mark_categorical_cols_for_transactions(df) for df in dfs]

In [15]:
hist_trans.shape, new_hist_trans.shape

((29112361, 29), (1963031, 29))

In [16]:
hist_trans.to_feather('hist_trans')
new_hist_trans.to_feather('new_hist_trans')

In [3]:
hist_trans = feather.read_dataframe('hist_trans')

In [4]:
new_hist_trans = feather.read_dataframe('new_hist_trans')

### One hot encoding categories (not working)

In [19]:
DataFrameSummary(new_hist_trans_c).summary()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,purchase_Dayofyear,purchase_Is_month_end,purchase_Is_month_start,purchase_Is_quarter_end,purchase_Is_quarter_start,purchase_Is_year_end,purchase_Is_year_start,purchase_Elapsed,purchased_on_weekend,month_diff
count,1.96303e+06,NaN,NaN,1.96303e+06,1.96303e+06,NaN,NaN,NaN,1.96303e+06,1.96303e+06,...,1.96303e+06,NaN,NaN,NaN,NaN,NaN,NaN,1.96303e+06,1.96303e+06,1.96303e+06
mean,1,NaN,NaN,0.0321421,0.682964,NaN,NaN,NaN,1.47652,-0.550969,...,109.884,NaN,NaN,NaN,NaN,NaN,NaN,1.51935e+09,0.307438,11.772
std,0,NaN,NaN,0.176378,1.58407,NaN,NaN,NaN,0.499448,0.694004,...,75.1097,NaN,NaN,NaN,NaN,NaN,NaN,6.6753e+06,0.461433,2.55921
min,1,NaN,NaN,0,-1,NaN,NaN,NaN,1,-0.746893,...,1,NaN,NaN,NaN,NaN,NaN,NaN,1.48834e+09,0,10
25%,1,NaN,NaN,0,0,NaN,NaN,NaN,1,-0.716629,...,71,NaN,NaN,NaN,NaN,NaN,NaN,1.51967e+09,0,10
50%,1,NaN,NaN,0,1,NaN,NaN,NaN,1,-0.674841,...,90,NaN,NaN,NaN,NaN,NaN,NaN,1.52146e+09,0,11
75%,1,NaN,NaN,0,1,NaN,NaN,NaN,2,-0.581616,...,112,NaN,NaN,NaN,NaN,NaN,NaN,1.52322e+09,1,12
max,1,NaN,NaN,1,999,NaN,NaN,NaN,2,263.157,...,365,NaN,NaN,NaN,NaN,NaN,NaN,1.52513e+09,1,23
counts,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,...,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031,1963031
uniques,1,290001,308,2,15,3,314,226129,2,75190,...,365,2,2,2,2,2,2,1667025,2,14


In [20]:
def one_hot_encode_dfs(df):
    return pd.get_dummies(df, columns=['category_2', 'category_3'])

In [21]:
hist_trans_c.shape, new_hist_trans_c.shape

((29112361, 29), (1963031, 29))

In [22]:
hist_trans_c_e, new_hist_trans_c_e = [one_hot_encode_dfs(df) for df in dfs]

In [23]:
hist_trans_c_e.shape, new_hist_trans_c_e.shape

((29112361, 35), (1963031, 35))

In [24]:
new_hist_trans_c_e.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount',
       'purchase_date', 'state_id', 'subsector_id', 'purchase_Year',
       'purchase_Month', 'purchase_Week', 'purchase_Day', 'purchase_Dayofweek',
       'purchase_Dayofyear', 'purchase_Is_month_end',
       'purchase_Is_month_start', 'purchase_Is_quarter_end',
       'purchase_Is_quarter_start', 'purchase_Is_year_end',
       'purchase_Is_year_start', 'purchase_Elapsed', 'purchased_on_weekend',
       'month_diff', 'category_2_1.0', 'category_2_2.0', 'category_2_3.0',
       'category_2_4.0', 'category_2_5.0', 'category_3_A', 'category_3_B',
       'category_3_C'],
      dtype='object')

In [25]:
## Not working
def aggregate_by_category(df):
    features = ['category_2_1.0', 'category_2_2.0', 'category_2_3.0',
       'category_2_4.0', 'category_2_5.0', 'category_3_A', 'category_3_B',
       'category_3_C']
    cat_agg = {"purchase_amount": ["sum", "mean"], "installments": ["sum", "mean"]}
    other_df = pd.DataFrame()
    for fe in features:
        g = df.groupby(['card_id', fe]).agg(cat_agg)
        g.columns = ['_'+fe+'_'.join(c).strip() for c in g.columns.values]
        g.reset_index(inplace=True)
        if other_df.empty:
            other_df = g
        else:
            other_df = pd.merge(other_df, g, on='card_id', how='left')
    return other_df

### Aggregate by card_id

In [13]:
# hist_trans.head().T

In [13]:
def aggregate_by_card_id(df):
    unique_cols = ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 
                   'subsector_id', 'purchase_Year', 'purchase_Month', 'purchase_Week', 'purchase_Day']
    aggs = {}
    for c in unique_cols:
        aggs[c] = ['nunique'] 
    aggs['purchase_amount'] = ['sum','max','min','mean','var']
    aggs['installments'] = ['sum','max','min','mean','var']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var']
    aggs['month_diff'] = ['mean', 'std', 'var']
    aggs['authorized_flag'] = ['sum', 'mean']
    aggs['purchased_on_weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']
    for col in ['category_2','category_3']:
        df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
        df[col+'_sum'] = df.groupby([col])['purchase_amount'].transform('sum')
        aggs[col+'_mean'] = ['mean']
        aggs[col+'_sum'] = ['sum']

    new_df = df.groupby(['card_id']).agg(aggs)
    new_df.columns = ['_'.join(col).strip() for col in new_df.columns.values]
    new_df.reset_index(inplace=True)
    other_df = (df.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    new_df = pd.merge(other_df, new_df, on='card_id', how='left')

    new_df['purchase_date_diff'] = (new_df['purchase_date_max'] - new_df['purchase_date_min']).dt.days
    new_df['purchase_date_average'] = new_df['purchase_date_diff']/new_df['card_id_size']
    new_df['purchase_date_uptonow'] = (datetime.datetime.today() - new_df['purchase_date_max']).dt.days
    return new_df

In [14]:
# dfs_e = [hist_trans_c_e, new_hist_trans_c_e]

In [15]:
import gc
gc.collect()

38

In [16]:
%time hist_trans_agg, new_hist_trans_agg = [aggregate_by_card_id(df) for df in dfs]

CPU times: user 2min 1s, sys: 16.7 s, total: 2min 17s
Wall time: 1min 17s


### Add exta interpreted columns on aggregates

In [17]:
def add_extra_cols_on_agg(df):
    df['inverse_avg_transactions_per_day'] = df['purchase_date_diff']/df['card_id_size']
    df['days_since_last_transaction'] = (datetime.datetime.today() - df['purchase_date_max']).dt.days
    df['repurchase_merchant_rate'] = df['transactions_count']/df['merchant_id_nunique']
    df['merchant_category_repurchase'] = df['merchant_category_id_nunique']/df['merchant_id_nunique']
    df['category_2_sum_sum'] = df['category_2_sum_sum']/df['transactions_count']
    df['category_3_sum_sum'] = df['category_3_sum_sum']/df['transactions_count']
    df['avg_spend_per_merchant'] = df['purchase_amount_sum']/df['merchant_id_nunique']
    df['avg_trans_per_merchant'] = df['transactions_count']/df['merchant_id_nunique']
    df['avg_spend_per_transaction'] = df['purchase_amount_sum']/df['transactions_count']
    return df

In [18]:
[hist_trans_agg, new_hist_trans_agg] = [add_extra_cols_on_agg(df) for df in [hist_trans_agg, 
                                                                             new_hist_trans_agg]]

In [19]:
hist_trans_agg.to_feather('hist_trans_agg')
new_hist_trans_agg.to_feather('new_hist_trans_agg')

In [20]:
hist_trans_agg = feather.read_dataframe('hist_trans_agg')
new_hist_trans_agg = feather.read_dataframe('new_hist_trans_agg')

### Load test & train DFs

In [21]:
PATH = 'data/elo/'

In [22]:
train, test = [pd.read_csv(f'{PATH}{c}') for c in ['train.csv', 'test.csv']]

### Add date part to test & train dfs

In [23]:
add_datepart(train, 'first_active_month', drop=False)
add_datepart(test, 'first_active_month', drop=False)

### Merge train & test with new & old transactions history

In [24]:
def join_dfs(left, right, left_on, right_on=None, suffix='_old'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, suffixes=("", suffix))

In [25]:
train_df = join_dfs(train, new_hist_trans_agg, left_on='card_id')

In [26]:
train_df = join_dfs(train_df, hist_trans_agg, left_on='card_id', suffix='_old')

In [27]:
test_df = join_dfs(test, new_hist_trans_agg, left_on='card_id')
test_df = join_dfs(test_df, hist_trans_agg, left_on='card_id', suffix='_old')

In [28]:
test_df.shape, train_df.shape

((123623, 118), (201917, 119))

### Add extra columns like age, total transactions, installments, purchase amount, first buy etc

In [29]:
for df in [train_df, test_df]:
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['card_id_total'] = df['card_id_size']+df['card_id_size_old']
    df['purchase_amount_total'] = df['purchase_amount_sum']+df['purchase_amount_sum_old']
    df['installments_total'] = df['installments_sum'] + df['installments_sum_old']
    df['hist_first_buy'] = (df['purchase_date_min_old'] - df['first_active_month']).dt.days
    df['new_first_buy'] = (df['purchase_date_min'] - df['first_active_month']).dt.days
    df['avg_spend_per_transaction'] = df['purchase_amount_total']/df['card_id_total']

### Mark the outliers

In [30]:
train_df['outliers'] = 0
train_df.loc[train_df['target'] < -30, 'outliers'] = 1
train_df['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

### Target encode the outliers 

In [31]:
train_df.shape, test_df.shape

((201917, 126), (123623, 124))

In [32]:
for f in ['feature_1','feature_2','feature_3']:
    order_label = train_df.groupby([f])['outliers'].mean()
    train_df[f] = train_df[f].map(order_label)
    test_df[f] = test_df[f].map(order_label)

In [33]:
train_df.to_feather('train_df')
test_df.to_feather('test_df')

In [38]:
# train_df.to_feather('train_df_3_696')
# test_df.to_feather('test_df_3_696')

In [39]:
# hist_trans_agg.to_feather('hist_trans_agg_3_696')
# new_hist_trans_agg.to_feather('new_hist_trans_agg_3_696')

In [40]:
# hist_trans.to_feather('hist_trans_3_696')
# new_hist_trans.to_feather('new_hist_trans_3_696')